Symulacja kasy
Stan kasy (nie kwota, a ilość różnych nominałów)


W pętli obsługa klientów
Jakie nominały daje klient
Wydawanie reszty (przy możliwie najmniejszej ilości nominałów)

Przejście do kolejnej obsługi klienta

Uniknąć wydawania nominałów, których nie posiadamy


Na 3 - zrobić przy pomocy algorytmu zachłannego 
Na 5 - lepszy algorytm (zastosowanie np. programowania dynamicznego)

Problemy
- problem struktur danych 
- algorytm 

Artmetyka (10 groszy to nie 1/10)

Na koniec pracy generowanie raportu. 

In [5]:
def wydaj_reszte(reszta, kasa):
    nominaly = sorted(kasa.keys())
    max_val = reszta + 1
    dp = [max_val] * (reszta + 1)
    dp[0] = 0
    wybor = [{} for _ in range(reszta + 1)]

    for nom in nominaly:
        max_ilosc = kasa[nom]
        ilo = 1
        ilosci = []

        while max_ilosc > 0:
            curr = min(ilo, max_ilosc)
            ilosci.append(curr)
            max_ilosc -= curr
            ilo *= 2

        for count in ilosci:
            wartosc = nom * count
            for i in range(reszta, wartosc - 1, -1):
                if dp[i - wartosc] + count < dp[i]:
                    dp[i] = dp[i - wartosc] + count
                    wybor[i] = wybor[i - wartosc].copy()
                    wybor[i][nom] = wybor[i].get(nom, 0) + count

    if dp[reszta] == max_val:
        return None

    return wybor[reszta]

def aktualizuj_kase(kasa, nominaly, dodaj=True):
    for nom, ilosc in nominaly.items():
        kasa.setdefault(nom, 0)
        if not dodaj and kasa[nom] < ilosc:
            raise ValueError(f"Brak wystarczającej liczby monet: {nom}")
        kasa[nom] += ilosc if dodaj else -ilosc


def zlicz_nominaly(lista_nominalow):
    zliczone = {}
    for nom in lista_nominalow:
        zliczone[nom] = zliczone.get(nom, 0) + 1
    return zliczone


def wyswietl_stan_kasy(kasa):
    print("\nAktualny stan kasy:")
    for nom, ilosc in sorted(kasa.items(), reverse=True):
        print(f"Nominał {nom / 100:.2f} zł: {ilosc} szt.")


def symuluj():
    kasa = {
        50000: 2, 20000: 3, 10000: 5, 5000: 10,
        2000: 20, 1000: 50, 500: 50, 200: 100,
        100: 100, 50: 200, 20: 200, 10: 200,
        5: 100, 2: 100, 1: 100
    }

    wyswietl_stan_kasy(kasa)

    while True:
        print("\nNowy klient")
        try:
            kwota_do_zaplaty = int(float(input("Podaj kwotę do zapłaty (np. 15.50): ")) * 100)
        except ValueError:
            print("Niepoprawna kwota, spróbuj ponownie.")
            continue

        while True:
            klient_nominaly_input = input("Jakie nominały daje klient? (np. 500zl,200zl): ").replace(' ', '').lower()
            klient_nominaly = []
            try:
                for nom in klient_nominaly_input.split(','):
                    if 'zl' in nom:
                        wartosc = int(float(nom.replace('zl', '')) * 100)
                        klient_nominaly.append(wartosc)
                    else:
                        raise ValueError
                break
            except ValueError:
                print("Wprowadzono niepoprawny nominał, spróbuj ponownie.")

        suma_klienta = sum(klient_nominaly)
        reszta = suma_klienta - kwota_do_zaplaty

        if reszta < 0:
            print("Klient dał za mało pieniędzy!")
            continue

        klient_dict = zlicz_nominaly(klient_nominaly)
        aktualizuj_kase(kasa, klient_dict, dodaj=True)

        if reszta == 0:
            print("Klient zapłacił odliczoną kwotę.")
        else:
            wydane = wydaj_reszte(reszta, kasa)
            if wydane:
                print("Wydaj resztę:")
                for nom, ilosc in sorted(wydane.items(), reverse=True):
                    print(f"{nom / 100:.2f} zł: {ilosc} szt.")
                try:
                    aktualizuj_kase(kasa, wydane, dodaj=False)
                except ValueError:
                    print("Wystąpił błąd przy odejmowaniu monet.")
            else:
                print("Nie można wydać reszty!")
                aktualizuj_kase(kasa, klient_dict, dodaj=False)

        wyswietl_stan_kasy(kasa)

        if input("Czy kolejny klient? (tak/nie): ").lower() != 'tak':
            break

    print("\n=== Raport końcowy ===")
    wyswietl_stan_kasy(kasa)


if __name__ == "__main__":
    symuluj()


Aktualny stan kasy:
Nominał 500.00 zł: 2 szt.
Nominał 200.00 zł: 3 szt.
Nominał 100.00 zł: 5 szt.
Nominał 50.00 zł: 10 szt.
Nominał 20.00 zł: 20 szt.
Nominał 10.00 zł: 50 szt.
Nominał 5.00 zł: 50 szt.
Nominał 2.00 zł: 100 szt.
Nominał 1.00 zł: 100 szt.
Nominał 0.50 zł: 200 szt.
Nominał 0.20 zł: 200 szt.
Nominał 0.10 zł: 200 szt.
Nominał 0.05 zł: 100 szt.
Nominał 0.02 zł: 100 szt.
Nominał 0.01 zł: 100 szt.

Nowy klient


Podaj kwotę do zapłaty (np. 15.50):  0.01
Jakie nominały daje klient? (np. 500zl,200zl):  500zl


Wydaj resztę:
200.00 zł: 2 szt.
50.00 zł: 1 szt.
20.00 zł: 2 szt.
5.00 zł: 1 szt.
2.00 zł: 2 szt.
0.50 zł: 1 szt.
0.20 zł: 2 szt.
0.05 zł: 1 szt.
0.02 zł: 2 szt.

Aktualny stan kasy:
Nominał 500.00 zł: 3 szt.
Nominał 200.00 zł: 1 szt.
Nominał 100.00 zł: 5 szt.
Nominał 50.00 zł: 9 szt.
Nominał 20.00 zł: 18 szt.
Nominał 10.00 zł: 50 szt.
Nominał 5.00 zł: 49 szt.
Nominał 2.00 zł: 98 szt.
Nominał 1.00 zł: 100 szt.
Nominał 0.50 zł: 199 szt.
Nominał 0.20 zł: 198 szt.
Nominał 0.10 zł: 200 szt.
Nominał 0.05 zł: 99 szt.
Nominał 0.02 zł: 98 szt.
Nominał 0.01 zł: 100 szt.


Czy kolejny klient? (tak/nie):  tak



Nowy klient


...


Nowy klient


Podaj kwotę do zapłaty (np. 15.50):  0.01
Jakie nominały daje klient? (np. 500zl,200zl):  500zl


Nie można wydać reszty!

Aktualny stan kasy:
Nominał 500.00 zł: 8 szt.
Nominał 200.00 zł: 0 szt.
Nominał 100.00 zł: 0 szt.
Nominał 50.00 zł: 0 szt.
Nominał 20.00 zł: 0 szt.
Nominał 10.00 zł: 0 szt.
Nominał 5.00 zł: 0 szt.
Nominał 2.00 zł: 0 szt.
Nominał 1.00 zł: 56 szt.
Nominał 0.50 zł: 194 szt.
Nominał 0.20 zł: 188 szt.
Nominał 0.10 zł: 200 szt.
Nominał 0.05 zł: 94 szt.
Nominał 0.02 zł: 88 szt.
Nominał 0.01 zł: 100 szt.


Czy kolejny klient? (tak/nie):  nie



=== Raport końcowy ===

Aktualny stan kasy:
Nominał 500.00 zł: 8 szt.
Nominał 200.00 zł: 0 szt.
Nominał 100.00 zł: 0 szt.
Nominał 50.00 zł: 0 szt.
Nominał 20.00 zł: 0 szt.
Nominał 10.00 zł: 0 szt.
Nominał 5.00 zł: 0 szt.
Nominał 2.00 zł: 0 szt.
Nominał 1.00 zł: 56 szt.
Nominał 0.50 zł: 194 szt.
Nominał 0.20 zł: 188 szt.
Nominał 0.10 zł: 200 szt.
Nominał 0.05 zł: 94 szt.
Nominał 0.02 zł: 88 szt.
Nominał 0.01 zł: 100 szt.
